In [1]:
# python 3.10.11
!pip install SentencePiece

In [2]:
!pip install transformers pandas sumy torch 

In [3]:
!conda env export > sumy_transformer.yml

In [4]:
# @title Предварительная чистка датасета

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = nltk.corpus.stopwords.words('russian')

def cleaning_text(text):
#     text = text.lower() # Convert to lowercase
    text = text.strip()
    text = re.sub('<[^>]+>', '',text ,flags = re.DOTALL)
    text = re.sub(r"\n", ". ", text)  # Replace ( · ) 
    text = re.sub(r"\s·", ". ", text)  # Replace ( · )
    text = re.sub(r"\W\W\W", ". ", text)  # Replace ( · )
    text = re.sub(r"\W\W", ". ", text)  # Replace ( · ) 
    text = re.sub("/[^\.\,\-\_\'\"\@\?\!\:\$ a-zA-Z0-9А-Яа-я()]/u" ,"", text)
    text = re.sub('\s\W', ' ', text)
    text = re.sub('Требования.', '', text)
    text = re.sub('Обязанности.', '', text)
    text = re.sub('\W,\s', '', text) # (Любая не-буква, не-цифра и не подчёркивание) (Любой пробельный символ)
    text = re.sub('\s+', ' ', text) # Любой пробельный символ
#     text = ' '.join([i for i in text.split() if i not in stop_words]) # Remove stop_words
    text = re.sub("nbsp", "", text)
    text = re.sub('\W\s\W', '.', text)
    text = re.sub('{', '', text)
    text = re.sub('}', '', text)
#     print("::::: Text_Cleaned :::::")
    return text


# text = recleaning_text(text_1)
# print(text)
# print(len(text))
# print(cleaning_text(text_3))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nikivene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nikivene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# @title Реализация через `sumy`
import nltk
import pandas as pd
import numpy as np
from string import punctuation
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from nltk.corpus import stopwords


def sumy_LexRank(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")    
    summarizerLexRank = LexRankSummarizer(stemmer)
    summary = ''
    for sentenceLexRank in summarizerLexRank(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLexRank)
    return summary


def sumy_Luhn(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerLuhn = LuhnSummarizer(stemmer)
    summary = ''
    for sentenceLuhn in summarizerLuhn(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLuhn)
    return summary

def sumy_LSA(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerLSA = LsaSummarizer(stemmer)
    summary = ''
    for sentenceLSA in summarizerLSA(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLSA)
    return summary

def sumy_TextRank(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerTR = TextRankSummarizer(stemmer)
    summary = ''
    for sentenceTextRank in summarizerTR(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceTextRank)
    return summary

def sumy_KLdiv(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerKL = KLSummarizer(stemmer)
    summary = ''
    for sentenceKL in summarizerKL(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceKL)
    return summary

In [6]:
def summarization_pipline_sumy_LexRank(text):
    text = cleaning_text(text)
    text = sumy_LexRank(text, sentences=3)
    return text

def summarization_pipline_sumy_Luhn(text):
    text = cleaning_text(text)
    text = sumy_Luhn(text, sentences=3)
    return text

def summarization_pipline_sumy_LSA(text):
    text = cleaning_text(text)
    text = sumy_LSA(text, sentences=3)
    return text

def summarization_pipline_sumy_TextRank(text):
    text = cleaning_text(text)
    text = sumy_TextRank(text, sentences=3)
    return text

def summarization_pipline_sumy_KLdiv(text):
    text = cleaning_text(text)
    text = sumy_KLdiv(text, sentences=3)
    return text

In [8]:
import pandas as pd

df_to_summary = pd.read_csv('./data/df_to_summary.csv')

df_to_summary.head()

,Unnamed: 0,full_text
0,0,"Обязанности: \r\n \r\n Приёмка, расценк..."
1,1,Обязанности: \r\n Приемка и контроль качеств...
2,2,Кадастровый инженер\r\n\r\nВ связи с расширени...
3,3,Ольга Гринюк (Olga Grinyuk) - российская марка...
4,4,Обязанности: \r\nПошив чехлов на мягкую м...


In [9]:
df_to_summary['sumy_LSA'] = df_to_summary['full_text'].apply(summarization_pipline_sumy_LSA)
df_to_summary['sumy_TextRank'] = df_to_summary['full_text'].apply(summarization_pipline_sumy_TextRank)
df_extract = df_to_summary
df_to_summary.head()

,Unnamed: 0,full_text,sumy_LSA,sumy_TextRank
0,0,"Обязанности: \r\n \r\n Приёмка, расценк...",выкладка товара. поддержание чистоты. Смена с...,выкладка товара. поддержание чистоты. Смена с...
1,1,Обязанности: \r\n Приемка и контроль качеств...,Приемка и контроль качества сырья. участие в ...,Приемка и контроль качества сырья. участие в ...
2,2,Кадастровый инженер\r\n\r\nВ связи с расширени...,В связи с расширением объема услуг. в агентст...,в агентство недвижимости требуется кадастровы...
3,3,Ольга Гринюк (Olga Grinyuk) - российская марка...,Вот уже 15 лет все платья создаются в Ростове...,Выпускается 12 коллекций в год. до 80 различн...
4,4,Обязанности: \r\nПошив чехлов на мягкую м...,Пошив чехлов на мягкую мебель. можно обучение...,Пошив чехлов на мягкую мебель. можно обучение...


In [10]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

MODEL_NAME = "cointegrated/rut5-base-paraphraser"

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
# model.cuda();
# model.eval();

def paraphrase(text, beams=3, grams=3, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams,
                         max_length=52, min_length=40,  do_sample=do_sample,
                         top_p=0.9)
    return tokenizer.decode(out[0], skip_special_tokens=True)

/home/nikivene/ITMO_Projects/Dataset_for_SFT/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def min_max_paraphrase(text):
    if type(text) == str:
        if 200 < len(text):
            return paraphrase(text)
            
    return np.NaN

In [12]:
df_extract['rut5-base-paraphraser(LSA)'] = df_extract['sumy_LSA'].apply(min_max_paraphrase)
df_extract['rut5-base-paraphraser(TextRank)'] = df_extract['sumy_TextRank'].apply(min_max_paraphrase)

In [13]:
df_abstract = df_extract[['rut5-base-paraphraser(LSA)','rut5-base-paraphraser(TextRank)']]

df_abstract.to_csv('./data/df_abstract.csv')
df_abstract.head(30)

,rut5-base-paraphraser(LSA),rut5-base-paraphraser(TextRank)
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,Готовы к развитию и совершенствованию своей пр...,Наш мастер зарабатывает от 3000 до 9000 рублей...
6,NaN,NaN
7,Разработчик и производитель специальных агрега...,NaN
8,NaN,NaN
9,NaN,NaN


In [14]:
df_extract.to_csv('summarization_final_100.csv', index=False)